## Summing all the prime numbers below a given number

In [10]:
import os
from time import perf_counter

# Parámetros (minúsculas en Python)
number = int(os.environ.get("NUMBER", "2500000"))  # default si no hay SLURM
nprocs = int(os.environ.get("NPROCS", str(os.cpu_count() or 1)))
print(f"number = {number} | nprocs = {nprocs}")

number = 2500000 | nprocs = 12


In [11]:
import time

# Simple code

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

number = 2_500_000
suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

The prime sum below  2500000 is  219697708195  and the time taken is 4.825361490249634
The prime sum below  2500000 is  219697708195  and the time taken is 4.58 s ± 1.68 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [12]:
tiempo = %timeit -r 2 -o -q sum_primes(number)
ssuma = sum_primes(number)

3) Numba secuencial (@njit)

In [13]:
import time
from numba import njit

@njit
def if_prime_numba(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes_numba(n):
    res = 0
    for i in range(n):
        res += if_prime_numba(i)
    return res

# Warmup para compilar
_ = sum_primes_numba(10000)

start = time.time()
suma = sum_primes_numba(number)
stop = time.time()

print("Numba secuencial → sum =", suma, "tiempo =", (stop - start))

Numba secuencial → sum = 219697708195 tiempo = 0.18276691436767578


4) multiprocessing.Pool usando nprocs

In [14]:
import time
from multiprocessing import Pool

def sum_primes_pool(n, nprocs):
    with Pool(processes=nprocs) as p:
        # chunksize ayuda a reducir overhead
        res = p.map(if_prime, range(n), chunksize=5000)
    return sum(res)

start = time.time()
suma = sum_primes_pool(number, nprocs)
stop = time.time()

print(f"Pool({nprocs}) → sum = {suma} | tiempo = {stop - start:.4f} s")

Pool(12) → sum = 219697708195 | tiempo = 1.0098 s


5) Numba paralela con prange

In [15]:
import time
from numba import njit, prange, set_num_threads, get_num_threads

@njit
def if_prime_numba(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit(parallel=True)
def sum_primes_numba_parallel(n):
    s = 0
    for i in prange(n):
        s += if_prime_numba(i)
    return s

# Ajusta hilos (si tienes nprocs definido)
# set_num_threads(nprocs)
# print("Numba threads:", get_num_threads())

# Warmup (compila)
_ = sum_primes_numba_parallel(10000)

start = time.time()
suma = sum_primes_numba_parallel(number)   # number debe estar definido antes
stop = time.time()

print("Numba parallel (prange) → sum =", suma, "tiempo =", (stop - start))

Numba parallel (prange) → sum = 219697708195 tiempo = 0.03367805480957031
